In [1]:
from linares_plot import *
import os
import datetime
import re


In [2]:
## seleccionar truco
T='Truco 1'
df=pd.read_excel(os.getcwd() + '\\ANOTACIONES2.xlsx', sheet_name= T )

In [3]:
df.head()

,tiempo,1a,1b,2a,2b,2c,3a,3b,4a,4b,...,34a,34b,35a,35b,36a,36b,37a,37b,38a,38b
0,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00:00:02,NaN,NaN,A7,NaN,NaN,a4,NaN,a4,NaN,...,a3,NaN,NaN,NaN,A5,NaN,NaN,NaN,A5,NaN
2,00:00:04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,a4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00:00:06,Se,NaN,NaN,NaN,NaN,O,NaN,NaN,NaN,...,L,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN
4,00:00:08,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O,NaN


In [4]:
list_columns_subjects = list(df.columns)
list_columns_subjects.remove('tiempo')

##list_columns_subjects=list_columns_subjects[70:84] ## para ir comprobando, una a una, que no haya espacios vacíos!

frames=[]

for subject in list_columns_subjects:
    #print(subject)
    dfs = df[['tiempo', subject]]
    dfs = dfs.loc[~pd.isna(dfs[subject])] # Quitar todos los NaN (qudarnos solo con los eventos)
    ####################################### Añadir una columna de segundos
    times_s=[]
    for i in range(0, len(dfs)):
        l = dfs.tiempo.iloc[i]
        times_s.append(  int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds()) )
    #
    dfs.columns=['tiempo', 'event']
    dfs['time_s'] = times_s
    ####################################### Añadir columna de subject y de truco
    dfs['trick'] = int(T.split(' ')[-1]) ## last character of the string is the number of the trick
    dfs['subject'] = int(re.findall('\d+', subject)[0]) ##separate leter from number
    frames.append(dfs)
    
    
dfs=pd.concat(frames)

In [82]:
df_x = dfs.loc[dfs['subject']==3]
df_x = df_x.sort_values(by=['time_s'])
df_x['duration']=1
df_x = df_x.reset_index(drop=True) 
df_x

,tiempo,event,time_s,trick,subject,duration
0,00:00:02,a4,2,1,3,1
1,00:00:06,O,6,1,3,1
2,00:00:14,S,14,1,3,1
3,00:00:22,O,22,1,3,1
4,00:00:26,R,26,1,3,1
5,00:00:28,O*,28,1,3,1
6,00:00:48,P,48,1,3,1
7,00:00:52,L,52,1,3,1
8,00:01:04,S,64,1,3,1
9,00:01:18,P,78,1,3,1


In [83]:
boole=[] ###bolean to get the final of actions
for i in range(len(df_x)):
    stri = df_x['event'].iloc[i]
    boole.append( stri[-1]=='*' )

boole=np.array(boole)    
    
finals = df_x[boole]
finals

,tiempo,event,time_s,trick,subject,duration
5,00:00:28,O*,28,1,3,1


In [84]:
for fi in range(len(finals)):
    event_final = finals.event.iloc[fi][:-1] ##without the astherisc
    time_final = finals.time_s.iloc[fi]
    #
    previous_event = df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final) ].iloc[-1] ##get the last one
    duration_event = time_final - previous_event.time_s
    ## Change the duration of the dataframe
    df_x.at[previous_event.name, 'duration'] = duration_event
    
##
df_x = df_x[~boole] ### remove the events with astherist

In [85]:
boole_apl=[] ###bolean to get the final of actions
for i in range(len(df_x)):
    stri = df_x['event'].iloc[i]
    if stri[0]=='a':
        boole_apl.append(True)
    elif stri[0]=='A':
        boole_apl.append(True)
    else:
        boole_apl.append(False)
    
boole_apl=np.array(boole_apl)    
    
aplauses = df_x[boole_apl]
aplauses

,tiempo,event,time_s,trick,subject,duration
0,00:00:02,a4,2,1,3,1
43,00:05:56,A7,356,1,3,1
45,00:06:18,A+6,378,1,3,1


In [86]:
for ap in range(len(aplauses)):
    event_ = aplauses.iloc[ap] 
    name_event = event_.event
    duration_app = int(re.findall('\d+', name_event)[0])
    evnt = re.split('(\d+)', name_event)[0]
    #
    df_x.at[event_.name, 'duration'] = duration_app
    df_x.at[event_.name, 'event'] = evnt
    

In [87]:
df_x

,tiempo,event,time_s,trick,subject,duration
0,00:00:02,a,2,1,3,4
1,00:00:06,O,6,1,3,1
2,00:00:14,S,14,1,3,1
3,00:00:22,O,22,1,3,6
4,00:00:26,R,26,1,3,1
6,00:00:48,P,48,1,3,1
7,00:00:52,L,52,1,3,1
8,00:01:04,S,64,1,3,1
9,00:01:18,P,78,1,3,1
10,00:01:18,S,78,1,3,1


In [74]:
event_.name

41

In [301]:
previous_event.name

136

In [286]:
df_x = df_x.reset_index()
df_x

ValueError: cannot insert level_0, already exists

In [281]:
df_x.iloc[previous_event.name, 'duration']=25

IndexError: single positional indexer is out-of-bounds

In [271]:
df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final), 'duration' ].iloc[-1] 

0

In [276]:
df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final)].iloc[-1].duration

0

In [274]:
df_x

,tiempo,event,time_s,trick,subject,duration
3,00:00:06,Se,6,1,1,0
5,00:00:10,Si,10,1,1,0
6,00:00:12,N,12,1,1,0
16,00:00:32,In,32,1,1,0
31,00:01:02,In,62,1,1,0
46,00:01:32,In,92,1,1,0
54,00:01:48,L,108,1,1,0
57,00:01:54,N*,114,1,1,0
57,00:01:54,L,114,1,1,0
61,00:02:02,N,122,1,1,0


In [255]:
previous_event = df_x.loc[(df_x['event']=='N') & (df_x['time_s']<114) ].iloc[-1]

tiempo     00:00:12
event             N
time_s           12
trick             1
subject           1
Name: 6, dtype: object

'A8'

In [37]:
l = df.tiempo.iloc[158### hacer columna de tiempo en segundos
times_s=[]
for i in range(0, len(df)):
    l = df.tiempo.iloc[158]
    times_s.append(  int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds()) )



df['time_s'] = times_s
df.head()]
print(l.hour)
print(l.minute)
print(l.second)

0
5
16


In [38]:
l = df.tiempo.iloc[158]
int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds())

316

In [11]:
from datetime import datetime

datetime.strptime(df.tiempo.iloc[0], "%H:%M:%S")

TypeError: strptime() argument 1 must be str, not datetime.time